In [ ]:
import os
import torch
import subprocess
print(subprocess.check_output("pip install mmcv-full==latest+torch{} -f https://download.openmmlab.com/mmcv/dist/index.html".format(torch.__version__), shell=True))
!git clone https://github.com/purpleFar/street-view-house-numbers-detection.git
os.chdir('street-view-house-numbers-detection/')
!pip install -r requirements/build.txt
!pip install -v -e .

In [2]:
!nvidia-smi #show gpu information

Thu Nov 26 18:32:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
from mmdet.apis import init_detector
import numpy as np
import torch
from mmcv.ops import RoIPool
from mmcv.parallel import collate, scatter

from mmdet.datasets.pipelines import Compose
import time

total_time = 0
test_dir = 'inference_demo'
config_file = "configs/faster_rcnn/faster_rcnn_r34_fpn_1x_coco.py"
model = init_detector(config_file, device="cuda:0")

def inference_detector(model, img):
    cfg = model.cfg
    device = next(model.parameters()).device  # model device
    # prepare data
    if isinstance(img, np.ndarray):
        # directly add img
        data = dict(img=img)
        cfg = cfg.copy()
        # set loading pipeline type
        cfg.data.test.pipeline[0].type = 'LoadImageFromWebcam'
    else:
        # add information into dict
        data = dict(img_info=dict(filename=img), img_prefix=None)
    # build the data pipeline
    test_pipeline = Compose(cfg.data.test.pipeline)
    data = test_pipeline(data)
    data = collate([data], samples_per_gpu=1)
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]
    else:
        for m in model.modules():
            assert not isinstance(
                m, RoIPool
            ), 'CPU inference with RoIPool is not supported currently.'
        # just get the actual data from DataContainer
        data['img_metas'] = data['img_metas'][0].data

    # forward the model
    with torch.no_grad():
        start = time.time()
        model(return_loss=False, rescale=True, **data)[0]
        delay = time.time()-start
    return delay

for _, _, files in os.walk(test_dir):
    num = len(files)
    for file in files:
        total_time += inference_detector(model, os.path.join(test_dir, file))
        
fps = num/total_time
t_per_loop = int(total_time*1000/num)
print('Run {} loops, use {:.4} seconds in model inference. (Not included images loading times)'.format(num,total_time))
print('fps:{:.4}, {}ms per loop'.format(fps,t_per_loop))

Run 50 loops, use 1.758 seconds in model inference. (Not included images loading times)
fps:28.45, 35ms per loop
